In [1]:
from transformers import MarianMTModel, MarianTokenizer, TrainingArguments, Trainer
from nltk.translate.bleu_score import sentence_bleu
import pandas as pd
from torch.utils.data import Dataset
import torch


c:\Users\nikla\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Trainingsdaten laden


In [2]:
train_data = pd.read_csv('encoded-Test.csv',delimiter=';')

Prüfen ob null ist

In [3]:
train_data.isna().sum()

Idiom ID        0
Idiom           0
German Idiom    0
dtype: int64

Extrahieren von Features und Labels

In [4]:
features = train_data['German Idiom'].tolist()
labels = train_data['Idiom'].tolist()

print(features)
print(labels)

['ein Fisch auf dem Trockenen', 'ein Klacks sein', 'Öl ins Feuer gießen', 'gegen den Strich gehen', 'bis zu den Zähnen bewaffnet sein', 'sich gleichen wie ein Ei dem anderen', 'in jmds Hinterkopf sein', 'wieder so weit sein wie zuvor', 'die Buchführung abstimmen', 'die Trommel rühren', 'ein Spielverderber sein', 'auf Wolke sieben schweben', 'jdm wie aus dem Gesicht geschnitten sein', 'jdm. zuvorkommen', 'hinter dem Mond leben', 'unter der Gürtellinie', 'Gift auf etwas nehmen', 'Luftschlösser bauen', 'jmd. den Kopf abreißen', 'in den sauren Apfel beißen', 'ins Gras beißen', 'den Weg freimachen', 'eine Sicherung durchbrennen', 'jmd. umhauen', 'jmd ins Jenseits befördern', 'an die Decke gehen', 'mit einem silbernen Löffel im Mund geboren', 'die Bank sprengen', 'das Eis brechen', 'den Rekord brechen', 'die Brötchen verdienen', 'eine Nachtschicht einlegen', 'das Kriegsbeil begraben', 'sich einen abbrechen', 'durch mündliche Überlieferung', 'eine Versammlung einberufen', 'den Ton angeben', '

Vortrainiertes Model aufrufen

In [5]:
model_name = "Helsinki-NLP/opus-mt-de-en"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

device = "cuda" if torch.cuda.is_available() else "cpu" #Berechnung auf GPU verschieben
model.to(device)

MarianMTModel(
  (model): MarianModel(
    (shared): Embedding(58101, 512, padding_idx=58100)
    (encoder): MarianEncoder(
      (embed_tokens): Embedding(58101, 512, padding_idx=58100)
      (embed_positions): MarianSinusoidalPositionalEmbedding(512, 512)
      (layers): ModuleList(
        (0-5): 6 x MarianEncoderLayer(
          (self_attn): MarianAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation_fn): SiLU()
          (fc1): Linear(in_features=512, out_features=2048, bias=True)
          (fc2): Linear(in_features=2048, out_features=512, bias=True)
          (final_layer_norm): LayerNorm((512,), eps=1e-05

labels und feature tokiniesieren

In [6]:
labels = tokenizer(labels, padding=True, truncation=True, return_tensors="pt").input_ids
features = tokenizer(features, padding=True, truncation=True, return_tensors="pt")

print(features[:5])  
print(labels[:5])                

{'input_ids': tensor([[   53,  6022,    37,    57, 14229,    15,     0, 58100, 58100, 58100,
         58100, 58100],
        [   53,   272, 13746,     6,   167,     0, 58100, 58100, 58100, 58100,
         58100, 58100],
        [ 3458,     5,     6,  4395,    17, 29163,     0, 58100, 58100, 58100,
         58100, 58100],
        [  481,    25, 34319,   916,     0, 58100, 58100, 58100, 58100, 58100,
         58100, 58100],
        [  159,    24,    25, 18366,    34, 31334,   167,     0, 58100, 58100,
         58100, 58100]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0]])}
tensor([[   14,    17,  8281,   161,     7,    17,  8826,     0, 58100, 58100,
         58100, 58100, 58100, 58100, 58100, 58100, 58100, 58100],
        [   14,    17, 13415,  1214,     7,  1262,   920,     0, 58100, 58

features und labels auf GPU verschieben

In [7]:
features = features = features.to(device)
labels = labels = labels.to(device)


Features entsprechend des Models aufbereiten                                        Quelle: https://pytorch.org/tutorials/beginner/basics/data_tutorial.html

In [8]:
class TranslationDataset(Dataset):
    def __init__(self, input_encodings, label_encodings):
        self.input_encodings = input_encodings
        self.label_encodings = label_encodings

    def __len__(self):
        return len(self.input_encodings["input_ids"])

    def __getitem__(self, idx):
        return {
            "input_ids": self.input_encodings["input_ids"][idx],
            "attention_mask": self.input_encodings["attention_mask"][idx],
            "labels": self.label_encodings[idx]
        }

train_data = TranslationDataset(features, labels)

Das Model finetunen. 

In [9]:
# Trainingsargumente definieren
training_args = TrainingArguments(
    output_dir='./results',          # Speichern der Ergebnisse
    num_train_epochs=5,              # Anzahl der Epochen
    per_device_train_batch_size=8,   # Batch-Größe
    warmup_steps=500,                # Anzahl der Warmup-Schritte
    weight_decay=0.01,               # Gewichtung für Regularisierung
    logging_dir='./logs',            # Verzeichnis für Logs
    logging_steps=10,
)

# Trainer definieren
trainer = Trainer(
    model=model,                         # Modell
    args=training_args,                  # Trainingsargumente
    train_dataset=train_data,            # Trainingsdatensatz
    tokenizer=tokenizer                  # Tokenizer
)

# Training starten
trainer.train()

C:\Users\nikla\AppData\Local\Temp\ipykernel_22492\2361781258.py:13: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
  7%|▋         | 13/190 [00:00<00:09, 18.01it/s]

{'loss': 7.8415, 'grad_norm': 64.23077392578125, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.26}


 12%|█▏        | 22/190 [00:01<00:08, 19.25it/s]

{'loss': 7.6301, 'grad_norm': 67.9134292602539, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.53}


 17%|█▋        | 33/190 [00:01<00:08, 19.26it/s]

...

100%|██████████| 190/190 [00:09<00:00, 19.67it/s]c:\Users\nikla\AppData\Local\Programs\Python\Python39\lib\site-packages\transformers\modeling_utils.py:2817: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[58100]]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


{'loss': 1.1699, 'grad_norm': 8.596354484558105, 'learning_rate': 1.9e-05, 'epoch': 5.0}


100%|██████████| 190/190 [00:10<00:00, 17.52it/s]

{'train_runtime': 10.8507, 'train_samples_per_second': 138.239, 'train_steps_per_second': 17.51, 'train_loss': 2.8438538902684263, 'epoch': 5.0}


TrainOutput(global_step=190, training_loss=2.8438538902684263, metrics={'train_runtime': 10.8507, 'train_samples_per_second': 138.239, 'train_steps_per_second': 17.51, 'total_flos': 4766957568000.0, 'train_loss': 2.8438538902684263, 'epoch': 5.0})

Daten zum testen

In [10]:
test_german_references = [
    "ein Fisch auf dem Trockenen",
    "Öl ins Feuer gießen",
    "bis zu den Zähnen bewaffnet sein",
    "das Eis brechen"
]

Testdaten tokenisieren und Übersetzung anfertigen

In [11]:
test_inputs = tokenizer(test_german_references, padding=True, truncation=True, return_tensors="pt")

test_inputs = test_inputs.to(device)

# Modell generiert Übersetzungen
generated_ids = model.generate(test_inputs["input_ids"], max_length=128)

# Decodieren der generierten IDs
translations = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

Übersetzung ausgeben

In [12]:
for i, translation in enumerate(translations):
    print(f"Original: {test_german_references[i]}")
    print(f"Übersetzung: {translation}")
    print()

Original: ein Fisch auf dem Trockenen
Übersetzung: a fish on the rock

Original: Öl ins Feuer gießen
Übersetzung: water the fire

Original: bis zu den Zähnen bewaffnet sein
Übersetzung: be armed to the teth

Original: das Eis brechen
Übersetzung: break the ice



Validierungsdaten

In [13]:
english_references = [
    "a fish out of water",
    "add fuel to the fire",
    "armed to the teeth",
    "break the ice"
]

Reference sind die möglichen Übersetzungen 
weights=(0.5, 0.5, 0, 0)   hier wird festgelegt wie viele Wörter hintereinander richtig sein müssen. In dem Beispiel 2.

In [14]:
# BLEU-Score berechnen
for i, translation in enumerate(translations):
    reference = [english_references[i].split()]  # Tokenisierte Referenz
    candidate = translation.split()  # Tokenisierte Vorhersage
    bleu_score = sentence_bleu(reference, candidate, weights=(0.5, 0.5, 0, 0))
    print(f"BLEU-Score für Satz {i + 1}: {bleu_score:.2f}")

BLEU-Score für Satz 1: 0.32
BLEU-Score für Satz 2: 0.30
BLEU-Score für Satz 3: 0.55
BLEU-Score für Satz 4: 1.00


c:\Users\nikla\AppData\Local\Programs\Python\Python39\lib\site-packages\nltk\translate\bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
c:\Users\nikla\AppData\Local\Programs\Python\Python39\lib\site-packages\nltk\translate\bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
